<a href="https://colab.research.google.com/github/kimcaprio/lifeofkaggle/blob/master/Baseline_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline 모델
## tabular 데이터를 다루는 캐클의 파이프라인
### 1. 데이터 전처리 -> 2. 피처엔지니어링 -> 3. 머신러닝 모델학습 -> 4. 테스트 데이터 예측 및 캐글 업로드


1.   데이터 전처리

*   제품 변수의 결측값을 0으로 대체. 제품 보유 여부에 대한 정보가 없으면, 해당 제품을 보유하지 않고 있지 않다고 가정.
*   훈련 데이터와 테스트 데이터를 통합. 훈련 데이터와 테스트 데이터는 날짜변수(fecha_dato)로 쉽게 구분 가능. 동일한 24개의 고객 변수를 공유하고 있으며, 테스트 데이터에 없는 24개의 제품 변수는 0으로 채움.
* 번주형, 수치형 데이터를 전처리. 변수형 데이터는 .factorize()통해 label encoding을 수행. 데이터 타입이 object로 표현되는 수치형 데이터에서는 .unique()를 통해 특이값들을 대체하거나 제거하고, 정수형 데이터로 변환.
*   추후 모델 학습에 사용할 변수 이름을 features 리스트에 미리 담는다.







In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/My Drive/CoLab/explorer_of_machine_learning/for_kaggle/.kaggle/" # put path for wherever you put it

In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb

np.random.seed(2018)

# 데이터를 불러온다
trn = pd.read_csv('/content/drive/My Drive/CoLab/explorer_of_machine_learning/ch02_santander/data/train_ver2.csv')
tst = pd.read_csv('/content/drive/My Drive/CoLab/explorer_of_machine_learning/ch02_santander/data/test_ver2.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
## 데이터 전처리
# 제품 변수를 별도로 저장
prods = trn.columns[24:].tolist()

# 제품 변수 결측값을 미리 0으로 대체
trn[prods] = trn[prods].fillna(0.0).astype(np.int8)

# 24개 제품 중 하나도 보유하지 않는 고객 데이터를 제거
no_product = trn[prods].sum(axis=1) == 0
trn = trn[~no_product]

# 훈련 데이터와 테스트 데이터를 통합. 테스트 데이터에 없는 제품 변수는 0으로 대체
for col in trn.columns[24:]:
  tst[col] = 0
df = pd.concat([trn, tst], axis=0)

# 학습에 사용할 변수를 담는 list
features = []

# 범주형 변수를 .factorize() 함수를 통해 label encoding 함(one-hot) -- 날짜 데이터는 제외 시켜야...
# 그래서 모든 object type 변수를 fatorize 해서는 안됨... categorical_cols = [col for col in trn.columns[:24] if trn[col].dtype in ['O']]
categorical_cols = ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp','canal_entrada', 'indfall', 'tipodom', 'nomprov', 'segmento']
print(categorical_cols)
for col in categorical_cols:
  df[col], _ = df[col].factorize(na_sentinel=-99)
features += categorical_cols

# 수치형 변수의 특이값과 결측값을 -99로 대체하고, 정수형으로 변환
df['age'].replace(' NA', -99, inplace=True)
df['age'] = df['age'].astype(np.int8)

df['antiguedad'].replace('     NA', -99, inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

df['renta'].replace('         NA', -99, inplace=True)
df['renta'].fillna(-99, inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

df['indrel_1mes'].replace('P', 5, inplace=True)
df['indrel_1mes'].fillna(-99, inplace=True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

#학습에 사용할 수치형 변수를 features에 추가
#features +=  [col for col in trn.columns[:24] if trn[col].dtype in ['int64', 'float64']]
features += ['age', 'antiguedad', 'renta', 'ind_nuevo', 'indrel', 'indrel_1mes', 'ind_actividad_cliente']
print(features)

['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'nomprov', 'segmento']
['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'nomprov', 'segmento', 'age', 'antiguedad', 'renta', 'ind_nuevo', 'indrel', 'indrel_1mes', 'ind_actividad_cliente']


2.   피처 엔지니어링
*   모델 학습에 사용할 파생변수를 생성.

In [5]:
df['fecha_alta'].head()

0    2015-01-12
1    2012-08-10
2    2012-08-10
3    2012-08-10
4    2012-08-10
Name: fecha_alta, dtype: object

In [0]:
# (피처 엔지니어링) 두 날짜 변수에서 연도와 월 정보를 추출
df['fecha_alta_month'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['fecha_alta_year'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['fecha_alta_month', 'fecha_alta_year']

df['ult_fec_cli_1t_month'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['ult_fec_cli_1t_year'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['ult_fec_cli_1t_month', 'ult_cli_1t_year']

# 그 외 변수의 결측값은 모두 -99로 변환
df.fillna(-99, inplace=True)

In [0]:
# (피처 엔지니어링) lag-1 데이터를 생성
# # 날짜를 숫자로 변환하는 함수. 2015-01-28 은 1, 2016-06-28은 18로 변환
def date_to_int(str_date):
  Y, M, D = [int(a) for a in str_date.strip().split("-")]
  int_date = (int(Y) - 2015) * 12 + int(M)
  return int_date

# 날짜를 숫자로 변환하여 int_date에 저장
df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

# 데이터를 복사하고, int_date 날짜에 1을 더하여 lag를 생성. 변수명에 _prev를 추가
df_lag = df.copy()
df_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col for col in df.columns]
df_lag['int_date'] += 1

# 원본 데이터와 lag 데이터를 ncodper와 int_date 기준으로 합침. lag 데이터의 int_date는 1이 밀려 있기 때문에, 저번 달의 제품 정보가 삽입.
df_trn = df.merge(df_lag, on=['ncodpers', 'int_date'], how='left')

# 메모리 효율을 위해 불필요한 변수르 메모리에서 제거.
#del df, df_lag

# 저번달의 제품 정보가 존재하지 않을 경우를 대비하여 0으로 대체
for prod in prods:
  prev = prod + '_prev'
  df_trn[prev].fillna(0, inplace=True)
df_trn.fillna(-99, inplace=True)

# lag-1 변수를 추가
features += [feature + '_prev' for feature in features]
features += [prod + '_prev' for prod in prods]



3.   교차검증


*   테스트 데이터 제외하고 주어진 train 데이터를 활용하여 교차검증 데이터로 활용
*   시계열 데이터의 경우, 테스트 데이터 중 마지막 날짜 또는 년, 월 을 교차검증 데이터로 분리하여 활용하는 것이 일반적. <- 미래를 예측하는 모델이기 때문






In [8]:
## 모델 학습
# 학습을 위하여 데이터를 훈련, 테스트용으로 분리
# 학습에는 2016-01-28 ~ 2016-04-28 epdlxjaks tkdyd, 검증에는 2016-05-28 epdlxjfmf tkdyd.
use_dates = ['2016-01-28', '2016-02-28', '2016-03-28', '2016-04-28', '2016-05-28']
trn = df_trn[df_trn['fecha_dato'].isin(use_dates)]
tst = df_trn[df_trn['fecha_dato'] == '2016-06-28']
#del df_trn

print("tst", tst)

# 훈련 데이터에서 신규 구매 건수만 추출
X = []
Y = []
for i, prod in enumerate(prods):
  prev = prod + '_prev'
  prX = trn[(trn[prod] == 1) & (trn[prev] == 0)]
  prY = np.zeros(prX.shape[0], dtype=np.int8) + i
  X.append(prX)
  Y.append(prY)
XY = pd.concat(X)
Y = np.hstack(Y)
XY['y'] = Y

#훈련, 검증 데이터로 분리
vld_date = '2016-05-28'
XY_trn = XY[XY['fecha_dato'] != vld_date]
XY_vld = XY[XY['fecha_dato'] == vld_date]

print("XY_trn", XY_trn)
print("XY_vld", XY_vld)


tst           fecha_dato  ...  ult_fec_cli_1t_year_prev
11091070  2016-06-28  ...                       0.0
11091071  2016-06-28  ...                       0.0
11091072  2016-06-28  ...                       0.0
11091073  2016-06-28  ...                       0.0
11091074  2016-06-28  ...                       0.0
...              ...  ...                       ...
12020680  2016-06-28  ...                       0.0
12020681  2016-06-28  ...                     -99.0
12020682  2016-06-28  ...                       0.0
12020683  2016-06-28  ...                       0.0
12020684  2016-06-28  ...                       0.0

[929615 rows x 104 columns]
XY_trn           fecha_dato  ncodpers  ...  ult_fec_cli_1t_year_prev   y
7658069   2016-01-28   1474324  ...                       0.0   1
7628180   2016-01-28   1432311  ...                     -99.0   2
7628198   2016-01-28   1432232  ...                     -99.0   2
7628482   2016-01-28   1432080  ...                     -99.0   2
762869



1.   xgboost 모델을 사용 하여 학습 모델 만들기
2.   주요 파라미터
*   max_depth : 트리 모델의 최대 깊이를 의미. 값이 높을 수록 더 복잡한 트리를 만들지만, 과적합의 원인이 될수 있음.
*   eta : 딥러닝에서의 learning rate 의미. 0과 1사이의 값을 가지며, 값이 너무 높으면 학습이 잘 안되고, 낮으면 학습에 시간이 오래 걸림.
*   colsample_bytree : 트리를 생성할 때 룬련 데이터에서 변수를 샘플링해주는 비율. 마찬가지로 과적합을 막아주는 방안. (딥러닝의 drop out 개념?) 보통 0.6 ~ 0.9
*   colsample_bylevel : 트리의 레벨 별로 훈련 데이터의 변수를 샘플링해주는 비율. 보통 0.6 ~ 0.9






In [9]:
## XGBoost 모델 parameter를 설정
param = {
  'booster': 'gbtree',
  'max_depth' : 8,
  'nthread' : 4,
  'num_class' : len(prods),
  'objective' : 'multi:softprob',
  'silent' : 1,
  'eval_metric' : 'mlogloss',
  'eta' : 0.1,
  'min_child_weight' : 10,
  'colsample_bytree' : 0.8,
  'colsample_bylevel' : 0.9,
  'seed' : 2018,
}

# 훈련, 검증 데이터를 XGBoost 형태로 변환
X_trn = XY_trn.as_matrix(columns=features)
Y_trn = XY_trn.as_matrix(columns=['y'])
dtrn = xgb.DMatrix(X_trn, label=Y_trn, feature_names=features)

X_vld = XY_vld.as_matrix(columns=features)
Y_vld = XY_vld.as_matrix(columns=['y'])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)

print(Y_vld)

# XBBoost 모델을 훈련 데이터로 학습
watch_list = [(dtrn, 'train'), (dvld, 'eval')]
model = xgb.train(param, dtrn, num_boost_round=1000, evals=watch_list, early_stopping_rounds=20)

import pickle
pickle.dump(model, open('/content/drive/My Drive/CoLab/explorer_of_machine_learning/ch02_santander/modle/xgb.baseline.pkl', 'wb'))
best_ntree_limit = model.best_ntree_limit

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


[[ 0]
 [ 2]
 [ 2]
 ...
 [23]
 [23]
 [23]]
[0]	train-mlogloss:2.6752	eval-mlogloss:2.68357
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:2.43959	eval-mlogloss:2.4521
[2]	train-mlogloss:2.26071	eval-mlogloss:2.27549
[3]	train-mlogloss:2.12697	eval-mlogloss:2.14326
[4]	train-mlogloss:2.01445	eval-mlogloss:2.03139
[5]	train-mlogloss:1.92002	eval-mlogloss:1.93765
[6]	train-mlogloss:1.84373	eval-mlogloss:1.86197
[7]	train-mlogloss:1.77481	eval-mlogloss:1.79374
[8]	train-mlogloss:1.71598	eval-mlogloss:1.73584
[9]	train-mlogloss:1.66264	eval-mlogloss:1.68283
[10]	train-mlogloss:1.61532	eval-mlogloss:1.63602
[11]	train-mlogloss:1.57299	eval-mlogloss:1.59415
[12]	train-mlogloss:1.53602	eval-mlogloss:1.55785
[13]	train-mlogloss:1.50103	eval-mlogloss:1.52306
[14]	train-mlogloss:1.46888	eval-mlogloss:1.49102
[15]	train-mlogloss:1.44007	eval-mlogloss:1.46248
[16]	train-mlogloss

In [0]:
def apk(actual, predicted, k=7, default=0.0):
    if len(predicted) > k:
        predicted = predicted[:k]
    #MAP@7 이므로, 최대 7개만 사용

    score = 0.0
    num_hits = 0.0

    for i , p in enumerate(predicted):
        #점수를 부여하는 조건은 다음과 같음 :
        # 예측 값이 정답에 있고 ('p in actual')
        # 예측 값이 중복이 아니면('p not in predicted[:i]')
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

        #정답 값이 공백일 경우, 무조건 0.0을 반환
        if not actual:
            return default

        #정답의 개수(len(actual))로 averate precision을 구한다
        return score / min(len(actual), k)

#list of list인 정답 값(actual)과 예측값(predicted)에서 고객별 Average Precision을 구하고, np.mean()을 통해 평균을 계산
def mapk(actual, predicted, k=7, default=0.0):
    return np.mean([apk(a,p,k,default) for a,p in zip(actual, predicted)])





5.   평가





In [11]:
#MAP@7 평가 척도를 위한 준비작업.
# 고객 식별 버놓를 추출한다.
vld = trn[trn['fecha_dato'] == vld_date]
ncodpers_vld = vld.as_matrix(columns=['ncodpers'])

print("ncodpers_vld : ", ncodpers_vld)
print("vld : ", vld)

# 검증 데이터에서 신규 구매를 구한다
for prod in prods:
  prev = prod + '_prev'
  padd = prod + '_add'
  vld[padd] = vld[prod] - vld[prev]
add_vld = vld.as_matrix(columns=[prod + '_add' for prod in prods])

print("add_vld : ",add_vld)

print("ncodpers_vld.len : ", len(ncodpers_vld))

add_vld_list = [list() for i in range(len(ncodpers_vld))]

print("add_vld_list.len : ", add_vld_list[0:10])

#import map7 as mapk
# 고객 데이터에서 얻을 수 있는 MAP@7 최고점을 미리 구함.(0.042663)
print(mapk(add_vld_list, add_vld_list, 7, 0.0))

# 검증 데이터에 대한 예측값을 구한다
X_vld = vld.as_matrix(columns=features)
Y_vld = vld.as_matrix(columns=['y'])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)
preds_vld = model.predict(dvld, ntree_limit=best_ntree_limit)

# 저번 달에 보유한 제품은 신규 구매가 불가하기 때문에, 확률값에서 미리 1을 빼줌
preds_vld = preds_vld - vld.as_matrix(columns=[prod + '_prev' for prod in prods])

# 검증 데이터 예측 상위 7개를 추출
result_vld = []
for ncodper, pred in zip(ncodpers_vld, preds_vld):
  y_prods = [(y,p,ip) for y, p, ip, in zip(pred, prods, range(len(prods)))]
  y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
  result_vld.append([ip for y,p,ip in y_prods])

# 검증 데이터에서의 MAP@7 점수를 구한다.
print(mapk(add_vld_list, result_vld, 7, 0.0))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


ncodpers_vld :  [[ 657788]
 [ 657795]
 [ 657790]
 ...
 [1166763]
 [1166789]
 [1550586]]
vld :            fecha_dato  ...  ult_fec_cli_1t_year_prev
10394531  2016-05-28  ...                       0.0
10394532  2016-05-28  ...                       0.0
10394533  2016-05-28  ...                       0.0
10394534  2016-05-28  ...                       0.0
10394535  2016-05-28  ...                       0.0
...              ...  ...                       ...
11091065  2016-05-28  ...                       0.0
11091066  2016-05-28  ...                       0.0
11091067  2016-05-28  ...                       0.0
11091068  2016-05-28  ...                       0.0
11091069  2016-05-28  ...                     -99.0

[696539 rows x 104 columns]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


add_vld :  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
ncodpers_vld.len :  696539
add_vld_list.len :  [[], [], [], [], [], [], [], [], [], []]


TypeError: ignored